# Stitching Figures Together for Zip Code
_Calvin Whealton_

This is a simple notebook that will stitch together all the plots generated for a given zip code. The general layout will be.

_Tends in Gages_ : Top Left

_Housing Values, Flood Insurance Claims, and Gage Return Periods_ : Vertical Panel on Top Right

_Housing Value Predictions_ : Bottom as Stacked Verical Panel

Not all figures are defined for each zip code. Layout may be variable.

In [ ]:
import os
from PIL import Image
import pandas as pd

In [ ]:
# stiching the figures together
# based on answer in: https://note.nkmk.me/en/python-pillow-concat-images/

def get_concat_v(zip_code):
    '''
    function to stitch all figures together for a zip code
    pass in the zip code as integer or string or float
    '''
    
    zip_use = str(int(zip_code)).zfill(5)
    
    # list of directories to check
    dirs = ['/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_gages',
           '/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_housing_vals',
           '/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_claims',
           '/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_rps',
           '/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_pred_100yr']
    
    endings = ['_zip_gage.png', '_housing.png','_zip_claims.png','_zip_rps.png','_Poor_market.png','_Neutral_market.png','_Typical_market.png','_Good_market.png']
    # widths and heights of figures
    widths = []
    heights = []
    in_dir = []
    
    ind = 0
    
    # check each folder to see if zip code is present
    for direct in dirs:
        
        # change directory
        os.chdir(direct)
    
        if zip_use+endings[ind] in os.listdir():
            in_dir.append(True)
            
            # load the image
            img_temp = Image.open(zip_use+endings[ind])
            widths.append(img_temp.width)
            heights.append(img_temp.height)
        else:
            in_dir.append(False)
            widths.append(0)
            heights.append(0)
            
        ind += 1
    
    # find dimension of stitched together image
    img_width = max(widths[0]+ widths[1], widths[-1])
    img_height = max(heights[0],sum(heights[1:4])) + 4*heights[-1]
    ind = 0
    
    # declare a new image
    dst = Image.new('RGB', (img_width, img_height),color='white')
    
    # coordinates
    x_coo = 0
    y_coo = 0
    
    # only if one of the figures was defined
    if True in in_dir:

        for direct in dirs:
            os.chdir(direct)

            if in_dir[ind] == True:

                # non zip housing prediction
                if ind == 0:
                    img_temp = Image.open(zip_use+endings[ind])

                    dst.paste(img_temp, (x_coo, y_coo))

                    # adjust coordinates
                    x_coo += img_temp.width
                    y_coo += 0

                elif ind <= 3:
                    img_temp = Image.open(zip_use+endings[ind])

                    dst.paste(img_temp, (x_coo, y_coo))

                    # adjust coordinates
                    y_coo += img_temp.height

                else:
                    y_coo = max(y_coo,heights[0])
                    x_coo = 0

                    # housing predictions
                    # four per zip code
                    for j in range(4):
                        img_temp = Image.open(zip_use+endings[ind+j])
                        dst.paste(img_temp, (x_coo, y_coo))

                        # adjust coordinates
                        x_coo += 0
                        y_coo += heights[-1]

            # increment index
            ind += 1

        # directory for saving file
        os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/pretty')

        # save the figure
        fig_name = zip_use + '_combined.png'
        dst.save(fig_name)
        dst.close()
        
    # nothing returned
    pass

In [ ]:
# housing data
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
housing = pd.read_csv('zillow_mon_pct_val.csv')

for z in housing['GEOID10_str'].values:
    get_concat_v(z)